In [2]:
%pip install -q kagglehub
print('done')

done


In [4]:
import amazon_dataset
X, y = amazon_dataset.load_amazon_reviews()

X = X.sample(n = 20000, random_state=1)
y = y.sample(n = 20000, random_state=1)
print(X)
print(y)

Dataset downloaded to: /kaggle/input/amazon-fine-food-reviews
Loading data from /kaggle/input/amazon-fine-food-reviews/Reviews.csv...
Original dataset shape: (568454, 10)
Shape after filtering out neutral reviews (Score=3): (525814, 10)
Data processing complete. Returning text reviews and binary sentiment labels.
87500     This is one of my favorite flavors.  This Fren...
476116    I bought this for my girl friend that recently...
225031    Not a big fan of chili actually... so these al...
155722    The tea helped me tremendously with getting a ...
55081     I use curry powder in a lot of my cooking - fr...
                                ...                        
227313    My son is on a very restricted diet, and this ...
400922    I like cats.  I like dogs.  I just don't like ...
45510     I was not sure what to expect with this drink ...
208236    I was surprised that these were flavored so po...
91683     The title says it all!!<br /><br />If you pay ...
Name: Text, Length: 20000

In [5]:
import pandas as pd
import numpy as np
import re
import string

#import dataset
#data = pd.read_csv("RateMyProfessor_Sample data.csv", usecols = ["comments", "student_star"])


#preprocessing
#remove all punctuation
X = X.str.replace(r'[^\w\s]+', '', regex = True)
#make all letters lowercase
X = X.str.lower()

print(X)

87500     this is one of my favorite flavors  this frenc...
476116    i bought this for my girl friend that recently...
225031    not a big fan of chili actually so these almon...
155722    the tea helped me tremendously with getting a ...
55081     i use curry powder in a lot of my cooking  fro...
                                ...                        
227313    my son is on a very restricted diet and this c...
400922    i like cats  i like dogs  i just dont like my ...
45510     i was not sure what to expect with this drink ...
208236    i was surprised that these were flavored so po...
91683     the title says it allbr br if you pay less you...
Name: Text, Length: 20000, dtype: object


In [6]:
#feature extraction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#list of stop words
stop_str = (['the', 'a', 'and', 'to', 'you', 'is', 'he', 'she', 'of', 'an', 'in',
            'but', 'his', 'her', 'was', 'have', 'with', 'take', 'with', 'that', 'do', 'be',
            'for', 'if', 'it', 'are', 'on', 'this', 'will', 'at', 'about', 'as', 'so', 'guy',
            'him', 'your', 'had', 'can', 'hes', 'from', 'me', 'its', 'shes', 'get', 'my'])

cv = CountVectorizer(stop_words = stop_str)
tfidf_transformer = TfidfTransformer()


#bag of words
X_bow = cv.fit_transform(X.values.astype('U'))
X_bow.shape

(20000, 40147)

In [7]:
#analysis of dataset
#class distribution, text length distribution, total unique words, most common words
import matplotlib.pyplot as plt

#print('The accuracy for Naive Bayes classifier using TF-IDF is {:.5f} on training data'.format(accuracy_score(y_pred = clf.predict(X_train), y_true = y_train)))
#pos_ct = sum(data['student_star'] > 0)
pos_ct = sum(y > 0)
#neg_ct = sum(data['student_star'] < 0)
neg_ct = sum(y == 0)

print('There are {:.0f} total records.'.format( pos_ct + neg_ct ))
print('There are {:.0f} positive records.'.format( pos_ct ))
print('There are {:.0f} negative records.'.format( neg_ct ))
print('{:.2f}% of records in the dataset are positive.\n'.format( (pos_ct / (pos_ct + neg_ct)) * 100 ))

'''
word_ct = pd.read_csv("RateMyProfessor_Sample data.csv", usecols = ["word_comment"])
plt.hist(word_ct, color='lightgreen', ec='black', bins=10)
plt.xlabel("Number of Words")
plt.ylabel("Frequency")
plt.title("Distribution of Word Count")
plt.show()
'''

print('\nThere are {:.0f} unique words.\n'.format( len(cv.get_feature_names_out()) ))

print('The 25 most common words, excluding stop words, are: ')
freqs = zip(cv.get_feature_names_out(), X_bow.sum(axis=0).tolist()[0])
# sort from largest to smallest
print( sorted(freqs, key=lambda x: -x[1]) [:25])

There are 20000 total records.
There are 16822 positive records.
There are 3178 negative records.
84.11% of records in the dataset are positive.


There are 40147 unique words.

The 25 most common words, excluding stop words, are: 
[('not', 10960), ('they', 9701), ('br', 9161), ('like', 8772), ('these', 8270), ('them', 7303), ('good', 6660), ('or', 6084), ('one', 6026), ('great', 5992), ('just', 5925), ('very', 5844), ('taste', 5639), ('all', 5592), ('coffee', 5427), ('product', 5098), ('we', 4952), ('when', 4788), ('flavor', 4773), ('tea', 4690), ('love', 4606), ('has', 4566), ('more', 4462), ('food', 4195), ('would', 4039)]


In [12]:
#training classifiers
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#train-test split with Bag of Words, then apply TF-IDF to have same set of sample data
X_bow_train, X_bow_test, y_train, y_test = train_test_split(X_bow, y, test_size = 0.2)

y_train_arr = y_train.to_numpy().reshape(-1)

X_tfidf_train = tfidf_transformer.fit_transform(X_bow_train)
X_tfidf_train.shape
X_tfidf_test = tfidf_transformer.fit_transform(X_bow_test)
X_tfidf_test.shape

print('Classifier\t Feature Extr.\t Train Acc.\t Test Acc.')
print('---------------------------------------------------------')

#naive bayes
from sklearn.naive_bayes import MultinomialNB
clf_bow = MultinomialNB().fit(X_bow_train, y_train_arr)
clf_tfidf = MultinomialNB().fit(X_tfidf_train, y_train_arr)
print('Naive Bayes\t BoW\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = clf_bow.predict(X_bow_train), y_true = y_train),
                                                              accuracy_score(y_pred = clf_bow.predict(X_bow_test), y_true = y_test)))
print('Naive Bayes\t TF-IDF\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = clf_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                                 accuracy_score(y_pred = clf_tfidf.predict(X_tfidf_test), y_true = y_test)))

print('---------------------------------------------------------')
#support vector machine
from sklearn.linear_model import SGDClassifier
svm_bow = SGDClassifier().fit(X_bow_train, y_train_arr)
svm_tfidf = SGDClassifier().fit(X_tfidf_train, y_train_arr)
print('SVM\t\t BoW\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svm_bow.predict(X_bow_train), y_true = y_train),
                                                          accuracy_score(y_pred = svm_bow.predict(X_bow_test), y_true = y_test)))
print('SVM\t\t TF-IDF\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svm_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                             accuracy_score(y_pred = svm_tfidf.predict(X_tfidf_test), y_true = y_test)))


print('---------------------------------------------------------')
#logistic regression
from sklearn.linear_model import LogisticRegression
lr_bow = LogisticRegression().fit(X_bow_train, y_train_arr)
lr_tfidf = LogisticRegression().fit(X_tfidf_train, y_train_arr)
print('Log Regression\t BoW\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = lr_bow.predict(X_bow_train), y_true = y_train),
                                               accuracy_score(y_pred = lr_bow.predict(X_bow_test), y_true = y_test)))
print('Log Regression\t TF-IDF\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = lr_tfidf.predict(X_tfidf_train), y_true = y_train),
                                               accuracy_score(y_pred = lr_tfidf.predict(X_tfidf_test), y_true = y_test)))

print('---------------------------------------------------------')
#linear support vector classification
from sklearn.svm import LinearSVC
svc_bow = LinearSVC(max_iter = 2000).fit(X_bow_train, y_train_arr)
svc_tfidf = LinearSVC().fit(X_tfidf_train, y_train_arr)
print('SVC\t\t BoW\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svc_bow.predict(X_bow_train), y_true = y_train),
                                               accuracy_score(y_pred = svc_bow.predict(X_bow_test), y_true = y_test)))
print('SVC\t\t TF-IDF\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svc_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                  accuracy_score(y_pred = svc_tfidf.predict(X_tfidf_test), y_true = y_test)))



Classifier	 Feature Extr.	 Train Acc.	 Test Acc.
---------------------------------------------------------
Naive Bayes	 BoW		 0.93650	 0.88800
Naive Bayes	 TF-IDF		 0.84088	 0.84275
---------------------------------------------------------
SVM		 BoW		 0.99600	 0.90250
SVM		 TF-IDF		 0.94881	 0.90850
---------------------------------------------------------
Log Regression	 BoW		 0.99187	 0.90950
Log Regression	 TF-IDF		 0.92344	 0.90200
---------------------------------------------------------
SVC		 BoW		 0.99981	 0.89700
SVC		 TF-IDF		 0.99150	 0.91750


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [9]:
#hyperparameter testing
from sklearn.model_selection import GridSearchCV
print('Testing for hyperparameters:')

#naive bayes
clf_param = {
    'fit_prior': (True, False),
    'force_alpha': (True, False),
    'alpha': (1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1),
}

gs_clf = GridSearchCV(clf_tfidf, clf_param, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(X_tfidf_train, y_train_arr)
print('Naive Bayes:')
for param_name in sorted(clf_param.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
print('---------------------------------------------------------')



#support vector machine
svm_param = {
    'max_iter': (1500, 1000, 500),
    'loss': ('hinge', 'perceptron', 'squared_error'),
    'alpha': (0.01, 0.005, 0.001),
}

gs_svm = GridSearchCV(svm_tfidf, svm_param, cv=5, n_jobs=-1)
gs_svm = gs_svm.fit(X_tfidf_train, y_train_arr)
print('SVM:')
for param_name in sorted(svm_param.keys()):
    print("%s: %r" % (param_name, gs_svm.best_params_[param_name]))
print('---------------------------------------------------------')



#logistic regression
lr_param = {
    'solver': ('lbfgs', 'liblinear', 'newton-cg'),
    'max_iter': (500, 750, 1000, 1250)
}

gs_lr = GridSearchCV(lr_tfidf, lr_param, cv=5, n_jobs=-1)
gs_lr = gs_lr.fit(X_tfidf_train, y_train_arr)
print('Logistic Regression:')
for param_name in sorted(lr_param.keys()):
    print("%s: %r" % (param_name, gs_lr.best_params_[param_name]))
print('---------------------------------------------------------')



#linear support vector classification
svc_param = {
    'max_iter': (500, 750, 1000, 2000, 2500),
    'C': (0.01, 0.1, 0.5, 1.0, 1.5),
}

gs_svc = GridSearchCV(svc_tfidf, svc_param, cv=5, n_jobs=-1)
gs_svc = gs_svc.fit(X_tfidf_train, y_train_arr)
print('SVM:')
for param_name in sorted(svc_param.keys()):
    print("%s: %r" % (param_name, gs_svc.best_params_[param_name]))
print('---------------------------------------------------------')

Testing for hyperparameters:
Naive Bayes:
alpha: 0.2
fit_prior: False
force_alpha: True
---------------------------------------------------------
SVM:
alpha: 0.005
loss: 'perceptron'
max_iter: 1500
---------------------------------------------------------
Logistic Regression:
max_iter: 500
solver: 'lbfgs'
---------------------------------------------------------
SVM:
C: 1.0
max_iter: 500
---------------------------------------------------------


In [19]:
#testing the optimized hyperparameters with TF-IDF
print('Results of applying tuned hyperparameters')
print('-----------------------------------------')

print('Classifier\tTrain Acc.\tTest Acc.')
clf_tfidf = MultinomialNB(alpha = 0.6).fit(X_tfidf_train, y_train_arr)
print('Naive Bayes\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = clf_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                                 accuracy_score(y_pred = clf_tfidf.predict(X_tfidf_test), y_true = y_test)))

svm_tfidf = SGDClassifier(alpha = 0.001, loss = 'squared_error', max_iter = 1500).fit(X_tfidf_train, y_train_arr)
print('SVM\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svm_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                             accuracy_score(y_pred = svm_tfidf.predict(X_tfidf_test), y_true = y_test)))

lr_tfidf = LogisticRegression(solver = 'liblinear', max_iter = 500).fit(X_tfidf_train, y_train_arr)
print('Log Regression\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = lr_tfidf.predict(X_tfidf_train), y_true = y_train),
                                               accuracy_score(y_pred = lr_tfidf.predict(X_tfidf_test), y_true = y_test)))

svc_tfidf = LinearSVC(C = 0.1, max_iter = 500).fit(X_tfidf_train, y_train_arr)
print('SVC\t\t {:.5f}\t {:.5f}'.format(accuracy_score(y_pred = svc_tfidf.predict(X_tfidf_train), y_true = y_train),
                                                  accuracy_score(y_pred = svc_tfidf.predict(X_tfidf_test), y_true = y_test)))

Results of applying tuned hyperparameters
-----------------------------------------
Classifier	Train Acc.	Test Acc.
Naive Bayes	 0.84506	 0.84300
SVM		 0.86638	 0.86850
Log Regression	 0.92344	 0.90200
SVC		 0.92375	 0.90275


In [22]:
from sklearn.metrics import accuracy_score, classification_report
#WITHOUT TUNING

# Logistic Regression
m_lr = LogisticRegression()
m_lr.fit(X_tfidf_train, y_train)
# Predictions
y_pred_lr_train = m_lr.predict(X_tfidf_train)
y_pred_lr_test = m_lr.predict(X_tfidf_test)

# Accuracy scores
lr_train_accuracy = accuracy_score(y_train, y_pred_lr_train)
lr_test_accuracy = accuracy_score(y_test, y_pred_lr_test)

# Print formatted output
print("Logistic Regression\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(lr_train_accuracy, lr_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_lr_test))



# Linear SVC
m_svc = LinearSVC()
m_svc.fit(X_tfidf_train, y_train)
# Predictions
y_pred_svc_train = m_svc.predict(X_tfidf_train)
y_pred_svc_test = m_svc.predict(X_tfidf_test)

# Accuracy scores
svc_train_accuracy = accuracy_score(y_train, y_pred_svc_train)
svc_test_accuracy = accuracy_score(y_test, y_pred_svc_test)

# Print formatted output
print("Linear SVC\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(svc_train_accuracy, svc_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_svc_test))



# Support vector machine
m_svm = SGDClassifier()
m_svm.fit(X_tfidf_train, y_train)
# Predictions
y_pred_svm_train = m_svm.predict(X_tfidf_train)
y_pred_svm_test = m_svm.predict(X_tfidf_test)

# Accuracy scores
svm_train_accuracy = accuracy_score(y_train, y_pred_svm_train)
svm_test_accuracy = accuracy_score(y_test, y_pred_svm_test)

# Print formatted output
print("Support Vector Machine\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(svm_train_accuracy, svm_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_svm_test))



# naive bayes
m_clf = MultinomialNB()
m_clf.fit(X_tfidf_train, y_train)
# Predictions
y_pred_clf_train = m_clf.predict(X_tfidf_train)
y_pred_clf_test = m_clf.predict(X_tfidf_test)

# Accuracy scores
clf_train_accuracy = accuracy_score(y_train, y_pred_clf_train)
clf_test_accuracy = accuracy_score(y_test, y_pred_clf_test)

# Print formatted output
print("Naive Bayes\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(clf_train_accuracy, clf_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_clf_test))

Logistic Regression	 Train Accuracy: 0.92344	 Test Accuracy: 0.90200
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.43      0.58       629
           1       0.90      0.99      0.94      3371

    accuracy                           0.90      4000
   macro avg       0.89      0.71      0.76      4000
weighted avg       0.90      0.90      0.89      4000

Linear SVC	 Train Accuracy: 0.99150	 Test Accuracy: 0.91750
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.63      0.71       629
           1       0.93      0.97      0.95      3371

    accuracy                           0.92      4000
   macro avg       0.87      0.80      0.83      4000
weighted avg       0.91      0.92      0.91      4000

Support Vector Machine	 Train Accuracy: 0.94819	 Test Accuracy: 0.90650
Classification Report:
               precision    recall  f1-score   support

           0      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [24]:
from sklearn.metrics import accuracy_score, classification_report
#WITH TUNING


# Logistic Regression
# Predictions
y_pred_lr_train = gs_lr.predict(X_tfidf_train)
y_pred_lr_test = gs_lr.predict(X_tfidf_test)

# Accuracy scores
lr_train_accuracy = accuracy_score(y_train, y_pred_lr_train)
lr_test_accuracy = accuracy_score(y_test, y_pred_lr_test)

print ("Logistic Regression")
for param_name in sorted(lr_param.keys()):
    print("%s: %r" % (param_name, gs_lr.best_params_[param_name]))
print('---------------------------------------------------------')

# Print formatted output
print("Logistic Regression\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(lr_train_accuracy, lr_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_lr_test))



# Linear SVC
# Predictions
y_pred_svc_train = gs_svc.predict(X_tfidf_train)
y_pred_svc_test = gs_svc.predict(X_tfidf_test)

# Accuracy scores
svc_train_accuracy = accuracy_score(y_train, y_pred_svc_train)
svc_test_accuracy = accuracy_score(y_test, y_pred_svc_test)

print ("Linear SVC")
for param_name in sorted(svc_param.keys()):
    print("%s: %r" % (param_name, gs_svc.best_params_[param_name]))
print('---------------------------------------------------------')

# Print formatted output
print("Linear SVC\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(svc_train_accuracy, svc_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_svc_test))



# Support vector machine
# Predictions
y_pred_svm_train = gs_svm.predict(X_tfidf_train)
y_pred_svm_test = gs_svm.predict(X_tfidf_test)

# Accuracy scores
svm_train_accuracy = accuracy_score(y_train, y_pred_svm_train)
svm_test_accuracy = accuracy_score(y_test, y_pred_svm_test)

print("Support Vector Machine")
for param_name in sorted(svm_param.keys()):
    print("%s: %r" % (param_name, gs_svm.best_params_[param_name]))
print('---------------------------------------------------------')

# Print formatted output
print("Support Vector Machine\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(svm_train_accuracy, svm_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_svm_test))



# naive bayes
# Predictions
y_pred_clf_train = gs_clf.predict(X_tfidf_train)
y_pred_clf_test = gs_clf.predict(X_tfidf_test)

# Accuracy scores
clf_train_accuracy = accuracy_score(y_train, y_pred_clf_train)
clf_test_accuracy = accuracy_score(y_test, y_pred_clf_test)

print("Naive Bayes")
for param_name in sorted(clf_param.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
print('---------------------------------------------------------')

# Print formatted output
print("Naive Bayes\t Train Accuracy: {:.5f}\t Test Accuracy: {:.5f}".format(clf_train_accuracy, clf_test_accuracy))

# print classification report
print("Classification Report:\n", classification_report(y_test, y_pred_clf_test))

Logistic Regression
max_iter: 500
solver: 'lbfgs'
---------------------------------------------------------
Logistic Regression	 Train Accuracy: 0.91731	 Test Accuracy: 0.92025
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.52      0.67       629
           1       0.92      0.99      0.95      3371

    accuracy                           0.92      4000
   macro avg       0.93      0.76      0.81      4000
weighted avg       0.92      0.92      0.91      4000

Linear SVC
C: 1.0
max_iter: 500
---------------------------------------------------------
Linear SVC	 Train Accuracy: 0.97637	 Test Accuracy: 0.97725
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.89      0.92       629
           1       0.98      0.99      0.99      3371

    accuracy                           0.98      4000
   macro avg       0.97      0.94      0.96      4000
weighted avg       0.98   